In [34]:
import pandas as pd

df = pd.read_csv('wine_weather.csv')


C:\Users\maxpi\AppData\Local\Temp\ipykernel_12388\2331004269.py:3: DtypeWarning: Columns (71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wine_weather.csv')


In [35]:
df.columns


Index(['name', 'country', 'region', 'winery', 'rating', 'num_ratings', 'price',
       'year', 'lat', 'lng', 'jan_tavg', 'jan_tmin', 'jan_tmax', 'jan_prcp',
       'jan_tsun', 'feb_tavg', 'feb_tmin', 'feb_tmax', 'feb_prcp', 'feb_tsun',
       'mar_tavg', 'mar_tmin', 'mar_tmax', 'mar_prcp', 'mar_tsun', 'apr_tavg',
       'apr_tmin', 'apr_tmax', 'apr_prcp', 'apr_tsun', 'may_tavg', 'may_tmin',
       'may_tmax', 'may_prcp', 'may_tsun', 'jun_tavg', 'jun_tmin', 'jun_tmax',
       'jun_prcp', 'jun_tsun', 'jul_tavg', 'jul_tmin', 'jul_tmax', 'jul_prcp',
       'jul_tsun', 'aug_tavg', 'aug_tmin', 'aug_tmax', 'aug_prcp', 'aug_tsun',
       'sep_tavg', 'sep_tmin', 'sep_tmax', 'sep_prcp', 'sep_tsun', 'oct_tavg',
       'oct_tmin', 'oct_tmax', 'oct_prcp', 'oct_tsun', 'nov_tavg', 'nov_tmin',
       'nov_tmax', 'nov_prcp', 'nov_tsun', 'dec_tavg', 'dec_tmin', 'dec_tmax',
       'dec_prcp', 'dec_tsun', 'type', 'city', 'country.1'],
      dtype='object')

Create region file and add continents column to it 

In [36]:
region_cols = ['region', 'country', 'lat', 'lng']
region_df = df[region_cols].drop_duplicates().reset_index(drop=True)

Add new column with continents using pycountry_convert

In [37]:

import pycountry_convert as pc

def country_to_continent(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return 'Unknown'

region_df['continent'] = region_df['country'].apply(country_to_continent)

In [38]:
region_df

,region,country,lat,lng,continent
0,Pomerol,France,44.931123,-0.199990,Europe
1,Lirac,France,44.037200,4.691360,Europe
2,Toscana,Italy,43.458654,11.138920,Europe
3,Bardolino,Italy,45.547559,10.724215,Europe
4,Carnuntum,Austria,48.114148,16.865783,Europe
...,...,...,...,...,...
833,Cape South Coast,South Africa,-29.000000,24.000000,Africa
834,Savennières-Coulée-de-Serrant,France,47.386976,-0.659663,Europe
835,Valdadige,Italy,40.732874,16.573639,Europe
836,Jurançon,France,43.287183,-0.389643,Europe


Climate table

In [39]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
          'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

climate_records = []

for i, month in enumerate(months, start=1):
    # df for each month with climate columns
    temp_df = df[['region']].copy()
    temp_df['month'] = i  # numeric month
    
    # get climate columns for each month
    temp_df['tavg'] = df[f'{month}_tavg']
    temp_df['tmin'] = df[f'{month}_tmin']
    temp_df['tmax'] = df[f'{month}_tmax']
    temp_df['prcp'] = df[f'{month}_prcp']
    temp_df['tsun'] = df[f'{month}_tsun']
    
    climate_records.append(temp_df)

# Concatenate all months into one DataFrame
climate_df = pd.concat(climate_records, ignore_index=True)

print(climate_df.head())

      region  month      tavg      tmin       tmax      prcp          tsun
0    Pomerol      1  3.215984  0.254694   6.495016  2.200000  16352.188477
1      Lirac      1  7.049790  3.608661  10.921564  0.987097  20302.472656
2    Toscana      1  7.390440  3.777000  11.025387  7.087097  15894.913086
3  Bardolino      1  6.341118  3.471629   9.411952  1.300000  22779.419922
4  Carnuntum      1  2.334078 -0.702145   4.994629  1.845161  13353.752930


In [40]:
climate_df

,region,month,tavg,tmin,tmax,prcp,tsun
0,Pomerol,1,3.215984,0.254694,6.495016,2.200000,16352.188477
1,Lirac,1,7.049790,3.608661,10.921564,0.987097,20302.472656
2,Toscana,1,7.390440,3.777000,11.025387,7.087097,15894.913086
3,Bardolino,1,6.341118,3.471629,9.411952,1.300000,22779.419922
4,Carnuntum,1,2.334078,-0.702145,4.994629,1.845161,13353.752930
...,...,...,...,...,...,...,...
157075,Marlborough,12,16.466511,12.336403,21.633179,4.058064,38551.261719
157076,Rueda,12,4.169382,0.314274,8.949759,1.112903,18953.107422
157077,Marlborough,12,16.466511,12.336403,21.633179,4.058064,38551.261719
157078,Marlborough,12,18.173969,12.646081,24.600918,1.719355,44898.128906


Create wine table 

In [41]:
wine_cols = ['name', 'winery', 'year', 'rating', 'num_ratings', 'price', 'type', 'region']
wine_df = df[wine_cols].drop_duplicates().reset_index(drop=True)


In [42]:
wine_df

,name,winery,year,rating,num_ratings,price,type,region
0,Pomerol 2011,Château La Providence,2011,4.2,100,95.00,Red,Pomerol
1,Lirac 2017,Château Mont-Redon,2017,4.3,100,15.50,Red,Lirac
2,Erta e China Rosso di Toscana 2015,Renzo Masi,2015,3.9,100,7.45,Red,Toscana
3,Bardolino 2019,Cavalchina,2019,3.5,100,8.72,Red,Bardolino
4,Ried Scheibner Pinot Noir 2016,Markowitsch,2016,3.9,100,29.15,Red,Carnuntum
...,...,...,...,...,...,...,...,...
13085,Private Bin Sauvignon Blanc 2019,Villa Maria,2019,4.1,4082,10.60,White,Marlborough
13086,Blanco (Verdejo) 2018,Marqués de Riscal,2018,3.7,4155,6.30,White,Rueda
13087,Sauvignon Blanc 2019,Oyster Bay,2019,4.0,4423,10.66,White,Marlborough
13088,Sauvignon Blanc 2018,Kim Crawford,2018,3.9,5105,14.90,White,Marlborough


In [43]:
wine_df.to_csv('wine_df2.csv', index=False)
climate_df.to_csv('climate_df.csv', index=False)
region_df.to_csv('region_df.csv', index=False)